# Tutorial (7): Useful functions

There are several useful functions to accelerate algorithm developments

[(0) Prepare and run UCCSD](#(0)-Prepare-and-run-UCCSD)  
[(1) Orbital Rotation](#(1)-Orbital-Rotation)  
[(2) Easy operator-state operation and `evolve()`](#(2)-Easy-operator-state-operation-and-evolve())  
 


In [1]:
# Import necessary modules
from quket import *
from quket.utils import *
import quket.config as cf

mpi4py is not imported. no MPI.


## (0) Prepare and run UCCSD
For this tutorial, we use first run SAUCCD-VQE

In [2]:
### Create QuketData for H2O ###
Q = create(basis="sto-3g", 
           ansatz="sauccsd", 
           n_orbitals =5, 
           n_electrons=6, 
           geometry = "O             ;\
                       H 1 1         ;\
                       H 1 1 2 110   "
        )
# Run UCCGD
Q.run()

Basis set = sto-3g

*** Geometry ******************************
  O     0.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    0.0000000
  H    -0.3420201    0.0000000    0.9396926
*******************************************

Symmetry C2v : C2v(Abelian)
E[FCI]    = -74.999082234384     (Spin = 1   Ms = 0)
E[HF]     = -74.961156000638     (Spin = 1   Ms = 0)


Overwritten attributes  contract_2e  of <class 'pyscf.fci.direct_spin1_symm.FCISolver'>


NBasis = 7
Entered VQE driver
Performing VQE for sauccsd
Number of VQE parameters: 27
Initial configuration: |0000111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccsd] = -74.961156000638  <S**2> = +0.000000  rho = 1  
      1: E[sauccsd] = -74.993183437851  <S**2> = +0.000000  Grad = 2.88e-01  CPU Time =    0.24779  (0.00 / step)
      2: E[sauccsd] = -74.997131045644  <S**2> = +0.000000  Grad = 1.43e-01  CPU Time =    0.08600  (0.00 / step)
      3: E[sauccsd] = -74.998746753620  <S**2> = +0.000001  Grad = 4.26e-02  CPU Time =    0.08225  (0.00 / step)
      4: E[sauccsd] = -74.998905435225  <S**2> = +0.000001  Grad = 3.17e-02  CPU Time =    0.08184  (0.00 / step)
      5: E[sauccsd] = -74.998993611443  <S**2> = +0.000001  Grad = 1.51e-02  CPU Time =    0.08677  (0.00 / step)
      6: E[sauccsd] = -74.999018526272  <S**2> = +0.000001  Grad = 4.23e-03  CPU Time =    0.08407  (0.00 / step)
      7: E[saucc

## (1) Orbital Rotation
In a different tutorial, we have seen `oo()` can perform orbital optimization.  
This function calls `orbital_rotation()`, which performs
\begin{equation}
\tilde H = e^{-\hat \kappa} \hat H e^{\hat \kappa}
\end{equation}
so $\tilde H$ replaces the original Hamiltonian $\hat H$.  
Running `orbital_rotation()` not only sets up the Fermionic Hamiltonian but also transform to the qubit Hamiltonian depending on the mapping, either Jordan-Wigner or Bravyi-Kitaev mapping.

Note that $\hat \kappa$ is a single particle operator,
\begin{equation}
\hat \kappa = \sum^{all\; orbitals}_{p>q} \kappa_{pq} (a^\dagger_p a_q - a^\dagger_q a_p)
\end{equation}
and is specified by `kappa_list`, which contains the lower-triangular part of $\kappa$ as a vector (i.e., only the $p>q$ part).  
Note that $p,q$ run over all orbitals (`n_orbitals` but not only `n_active_orbitals`).  

Here we show some example with random $\kappa$.

In [3]:
import numpy as np

# set random skew K
K = np.random.rand(5,5)
K = K - K.T

# append frozen occupied and virtual space to generate the full size matrix (we have 2 frozen core orbitals in this case)
Kfull = np.zeros((Q.n_orbitals, Q.n_orbitals))
Kfull[Q.n_frozen_orbitals+Q.n_core_orbitals:Q.n_frozen_orbitals+Q.n_core_orbitals+Q.n_active_orbitals,
     Q.n_frozen_orbitals+Q.n_core_orbitals:Q.n_frozen_orbitals+Q.n_core_orbitals+Q.n_active_orbitals] = K

# use vectorize_skew to get the vector form
from quket.linalg.linalg import vectorize_skew
kappa_list = vectorize_skew(Kfull)

In [4]:
printmat(Kfull, 'Full kappa matrix')
printmat(kappa_list, 'Vectorized skew kappa')


Full kappa matrix

               0              1              2              3              4              5              6          

    0      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    1      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    2      0.0000000      0.0000000      0.0000000      0.4073729      0.3758558      0.2646065      0.8551396  
    3      0.0000000      0.0000000     -0.4073729      0.0000000     -0.1017370     -0.4981962     -0.1422431  
    4      0.0000000      0.0000000     -0.3758558      0.1017370      0.0000000     -0.2090364     -0.9632859  
    5      0.0000000      0.0000000     -0.2646065      0.4981962      0.2090364      0.0000000      0.1216058  
    6      0.0000000      0.0000000     -0.8551396      0.1422431      0.9632859     -0.1216058      0.0000000  


Vectorized skew kappa
               0          
    0      0.0000000

### Copy the QuketData and perform `orbital_rotation()`

In [5]:
Qnew = Q.copy()
Qnew.orbital_rotation(kappa_list)

#### Check the Hamiltonians defined in Q and Qnew are different.

In [6]:
print(Q.operators.Hamiltonian - Qnew.operators.Hamiltonian)

() 0.0
((0, 1), (0, 0)) -0.017560713583547383
((0, 1), (2, 0)) 0.5056246753548188
((0, 1), (4, 0)) 0.46742766577312794
((0, 1), (6, 0)) 0.22705404747023825
((0, 1), (8, 0)) 0.9746500793872905
((1, 1), (1, 0)) -0.017560713583547383
((1, 1), (3, 0)) 0.5056246753548188
((1, 1), (5, 0)) 0.46742766577312794
((1, 1), (7, 0)) 0.22705404747023825
((1, 1), (9, 0)) 0.9746500793872905
((2, 1), (0, 0)) 0.5056246753548177
((2, 1), (2, 0)) -0.47187471336535536
((2, 1), (4, 0)) -0.15509849143625343
((2, 1), (6, 0)) -0.04424057663652137
((2, 1), (8, 0)) 0.07109746493086695
((3, 1), (1, 0)) 0.5056246753548177
((3, 1), (3, 0)) -0.47187471336535536
((3, 1), (5, 0)) -0.15509849143625343
((3, 1), (7, 0)) -0.04424057663652137
((3, 1), (9, 0)) 0.07109746493086695
((4, 1), (0, 0)) 0.46742766577312744
((4, 1), (2, 0)) -0.15509849143625304
((4, 1), (4, 0)) -0.9286967726012407
((4, 1), (6, 0)) 0.3143358167712712
((4, 1), (8, 0)) -0.5661484783401707
((5, 1), (1, 0)) 0.46742766577312744
((5, 1), (3, 0)) -0.1550984

### Confirm by energy
We can also confirm the result by generating a quantum state
\begin{equation}
|\tilde \Phi\rangle = e^{\hat \kappa} |\Phi\rangle
\end{equation}
and testing its energy is the same as $\langle \Phi| \tilde H |\Phi\rangle$.

In [7]:
# Use the UCCSD state obtained by VQE
state = Qnew.state.copy()
print_state(state, threshold=1e-3, name='UCCSD state')
print('<Phi| Htilde | Phi> = ', Qnew.get_E(state))

UCCSD state
     Basis             Coef
| 0000111111 > : +0.9876 +0.0000i
| 0011110011 > : -0.0490 +0.0000i
| 0011111100 > : -0.0479 +0.0000i
| 0110110110 > : +0.0538 +0.0000i
| 0110111001 > : -0.0382 +0.0000i
| 1001110110 > : -0.0382 +0.0000i
| 1001111001 > : +0.0538 +0.0000i
| 1100110011 > : -0.0473 +0.0000i
| 1100111100 > : -0.0791 +0.0000i

<Phi| Htilde | Phi> =  -73.4081966447339


`Decompose_expT1_to_Givens_rotations()` generates `pauli_list` and `theta_list` required to perform $e^{\hat \kappa}$ as a quantum circuit

In [8]:
from quket.opelib.opelib import Decompose_expT1_to_Givens_rotations
pauli_list, theta_list = Decompose_expT1_to_Givens_rotations(K)

`create_exp_state()` generates a quantum state by constructing a circuit with `pauli_list` and `theta_list`.

In [9]:
kappa_state = create_exp_state(Qnew, init_state = state, pauli_list=pauli_list, theta_list=theta_list)
print_state(kappa_state,  name='kappa_state')

kappa_state
     Basis             Coef
| 0100101111 > : -0.1534 +0.0000i
| 0110001111 > : -0.1914 +0.0000i
| 0110100111 > : +0.1312 +0.0000i
| 1000011111 > : +0.1534 +0.0000i
| 1001001111 > : +0.1914 +0.0000i
| 1001011011 > : +0.1312 +0.0000i
| 1100001111 > : +0.4767 +0.0000i
| 1100011011 > : +0.1273 +0.0000i
| 1100011110 > : -0.2318 +0.0000i
| 1100100111 > : -0.1273 +0.0000i
| 1100101101 > : +0.2318 +0.0000i
| 1101001011 > : -0.2780 +0.0000i
| 1101011010 > : -0.1528 +0.0000i
| 1101100011 > : +0.1005 +0.0000i
| 1101101001 > : +0.1530 +0.0000i
| 1110000111 > : +0.2780 +0.0000i
| 1110010011 > : -0.1005 +0.0000i
| 1110010110 > : +0.1530 +0.0000i
| 1110100101 > : -0.1528 +0.0000i
| 1111000011 > : +0.1080 +0.0000i



Evaluate the energy expectation value using the __*untransformed*__ Hamiltonian, preserved in `Q`.

In [10]:
print('<Phi e^{-k} |  H  | e^{k} Phi> = ', Q.get_E(kappa_state))

<Phi e^{-k} |  H  | e^{k} Phi> =  -73.40819664473402


You should have obtained the same energy.

Caution: Performing `initialize()` after `orbital_rotation()` will result in the original Hamiltonian computed with the Hartree-Fock state.

# (2) Easy operator-state operation and evolve()
In some cases, one may want to apply an arbitrary non-unitary operator, for example $\hat H$, to a quantum state.  
Such an operation is not trivial with quantum computers but the state-vector implementation of Quket allows us to do it.  

Quket extends the usabilities of `openfermion`'s `FermionOperator` and `QubitOperator` such that they can be directly applied to `Qulacs`' `QuantumState`.  
It also adds some easy-manipulations for `QuantumState` so that it can be added or subtracted.  

### Applying H to UCCSD state
Applying an operator (or scalar value) to a qunatum state can be performed by `*`

In [11]:
H = Q.operators.qubit_Hamiltonian
psi = Q.state.copy()
E = Q.energy

Hpsi = H * psi
Epsi = E * psi

The resulting instance is again `QuantumState`.

In [12]:
print_state(Hpsi)

     Basis             Coef
| 0000111111 > : -74.0684 +0.0000i
| 0001111011 > : +1.7937 +0.0000i
| 0010110111 > : -1.7937 +0.0000i
| 0011001111 > : +2.1270 +0.0000i
| 0011110011 > : +3.6779 +0.0000i
| 0011111100 > : +3.5892 +0.0000i
| 0101111010 > : +1.1945 +0.0000i
| 0110110110 > : -4.0335 +0.0000i
| 0110111001 > : +2.8663 +0.0000i
| 0111110010 > : -0.1093 +0.0000i
| 1001110110 > : +2.8663 +0.0000i
| 1001111001 > : -4.0335 +0.0000i
| 1010110101 > : +1.1945 +0.0000i
| 1011110001 > : +0.1093 +0.0000i
| 1100001111 > : +0.9975 +0.0000i
| 1100110011 > : +3.5462 +0.0000i
| 1100111100 > : +5.9291 +0.0000i
| 1101111000 > : -0.1525 +0.0000i
| 1110110100 > : +0.1525 +0.0000i
| 1111000011 > : -0.1512 +0.0000i
| 1111001100 > : -0.2193 +0.0000i
| 1111110000 > : -0.8176 +0.0000i



In [13]:
print_state(Epsi)

     Basis             Coef
| 0000111111 > : -74.0684 +0.0000i
| 0001111011 > : +1.7934 -0.0000i
| 0010110111 > : -1.7934 +0.0000i
| 0011001111 > : +2.1270 -0.0000i
| 0011110011 > : +3.6781 -0.0000i
| 0011111100 > : +3.5893 -0.0000i
| 0101111010 > : +1.1945 -0.0000i
| 0110110110 > : -4.0333 +0.0000i
| 0110111001 > : +2.8665 -0.0000i
| 0111110010 > : -0.1165 +0.0000i
| 1001110110 > : +2.8665 -0.0000i
| 1001111001 > : -4.0333 +0.0000i
| 1010110101 > : +1.1945 -0.0000i
| 1011110001 > : +0.1165 -0.0000i
| 1100001111 > : +0.9975 -0.0000i
| 1100110011 > : +3.5463 -0.0000i
| 1100111100 > : +5.9292 -0.0000i
| 1101111000 > : -0.1510 +0.0000i
| 1110110100 > : +0.1510 -0.0000i
| 1111000011 > : -0.1513 +0.0000i
| 1111001100 > : -0.2186 +0.0000i
| 1111110000 > : -0.8159 +0.0000i



Check how close Schrödinger equation is satisfied:

In [14]:
residual = Hpsi - Epsi
print(residual.get_squared_norm())

0.0001422010022550528


The UCCSD state `psi` is therefore very much satisfactory.  

If the mapping of `psi` is __Jordan-Wigner__, the same operation can be performed with `FermionOperator`.  
(For the use of Bravyi-Kitaev mapping and `FermionOperator`, see `evolve()`)

In [15]:
H_fermi = get_fermion_operator(Q.operators.Hamiltonian ) ### Q.operators.Hamiltonian is InteractionOperator but not FermionOperator, so transform
Hpsi_fermi = H_fermi * psi
Hpsi_fermi -= Hpsi
print(Hpsi_fermi.get_squared_norm())

2.5106642797332868e-26


#### `QuntumState`, `QubitOperator`, `FermionOperator` used in `Quket` are different from those with original libaries. 
These classes are overridden and defined in `quket.lib` module to enable these functionalities.  

In [16]:
import quket, openfermion, qulacs
print('QuantumState')
print(quket.QuantumState)
print(qulacs.QuantumState)

print('\nQubitOperator')
print(quket.QubitOperator)
print(openfermion.QubitOperator)

print('\nFermionOperator')
print(quket.FermionOperator)
print(openfermion.FermionOperator)

QuantumState
<class 'quket.lib.qulacs.QuantumState'>
<class 'qulacs.QuantumState'>

QubitOperator
<class 'quket.lib.openfermion.QubitOperator'>
<class 'openfermion.ops.operators.qubit_operator.QubitOperator'>

FermionOperator
<class 'quket.lib.openfermion.FermionOperator'>
<class 'openfermion.ops.operators.fermion_operator.FermionOperator'>


#### In order to use these features, the following functions should be replaced by those of Quket.

```python
from quket import(
    QubitOperator,
    FermionOperator,
    jordan_wigner,
    reverse_jordan_wigner,
    bravyi_kitaev,
    get_fermion_operator,
    commutator,
    s_squared_operator,
    number_operator,
    normal_ordered,
    hermitian_conjugated
    )
```

In [17]:
# Example:
n = quket.bravyi_kitaev(quket.number_operator(3))
print(n)
print(type(n))

(1.5+0j) [] +
(-0.5+0j) [Z0] +
(-0.5+0j) [Z0 Z1] +
(-0.5+0j) [Z2]
<class 'quket.lib.openfermion.QubitOperator'>


### evolve()
For a better peformance, Quket is equipped with `evolve()`, which applies an operator `A` to a quantum state `psi` in the same manner as `A*psi` (this operation actually calls `evolve()`), but with some advanced options.  

 - It accepts either `FermionOperator`, `QubitOperator`, or a list of them.  
 - `parallel=True` enables MPI parallelization.
 - `mapping='bk' or 'bravyi_kitaev'` allows to use a Bravyi-Kitaev-mapped quantum state with `FermionOperator`.  

The usage can be displayed by 
```
?evolve
```

In [18]:
Hpsi_evolve = quket.evolve(H, psi)
Hpsi_evolve1 = quket.evolve(H_fermi, psi, mapping='jw')

In [19]:
# They are the same
(Hpsi_evolve - Hpsi_evolve1).get_squared_norm()

2.5106642797332868e-26

In [20]:
# In the Bravyi-Kitaev mapping...
psi_bk = transform_state_jw2bk(psi)

Q.set(mapping='bk')
Q.initialize()

Hpsi_evolve_bk = quket.evolve(Q.operators.qubit_Hamiltonian, psi_bk)

Basis set = sto-3g

*** Geometry ******************************
  O     0.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    0.0000000
  H    -0.3420201    0.0000000    0.9396926
*******************************************

Symmetry C2v : C2v(Abelian)
E[FCI]    = -74.999082234384     (Spin = 1   Ms = 0)
E[HF]     = -74.961156000638     (Spin = 1   Ms = 0)


In [21]:
residual_bk = Hpsi_evolve_bk - E * psi_bk
print(residual_bk.get_squared_norm())

0.00014220100225505295


This is of course the same as the result above with the Jordan-Wigner mapping.